Problem Statement : Development of an NLP Model for Text Analytics and
Classification
• Objective:
o To develop an NLP model that categorizes complaint based on victim, type 
of fraud and other relevant parameters used for text classification and 
preparing the final model

In [1]:
import nltk
from nltk.probability import FreqDist
import pandas as pd
train_fraud=pd.read_csv("train.csv")
test_fraud=pd.read_csv("test.csv")

In [2]:
train_fraud['crimeaditionalinfo']=train_fraud['crimeaditionalinfo'].str.lower()

In [3]:
train_fraud.dropna()

,category,sub_category,crimeaditionalinfo
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,i had continue received random calls and abusi...
1,Online Financial Fraud,Fraud CallVishing,the above fraudster is continuously messaging ...
2,Online Gambling Betting,Online Gambling Betting,he is acting like a police and demanding for m...
3,Online and Social Media Related Crime,Online Job Fraud,in apna job i have applied for job interview f...
4,Online Financial Fraud,Fraud CallVishing,i received a call from lady stating that she w...
...,...,...,...
93681,Online Financial Fraud,Internet Banking Related Fraud,identity theft smishing sms fraud creditdeb...
93682,Online Financial Fraud,EWallet Related Fraud,received call from number asking about phone ...
93683,Online Financial Fraud,UPI Related Frauds,cyber stalking blackmailing phonesmsvoip c...
93684,Online and Social Media Related Crime,Online Matrimonial Fraud,call karke bola ki aapka lotary laga ha aru ac...


In [4]:


# Apply the function to each row in the 'crimeaditionalinfo' column
train_fraud['word_frequencies'] = train_fraud['crimeaditionalinfo'].dropna().apply((lambda x: x.split()))

# Display the DataFrame with per-row word frequencies
print(train_fraud[['crimeaditionalinfo', 'word_frequencies']])

                                      crimeaditionalinfo  \
0      i had continue received random calls and abusi...   
1      the above fraudster is continuously messaging ...   
2      he is acting like a police and demanding for m...   
3      in apna job i have applied for job interview f...   
4      i received a call from lady stating that she w...   
...                                                  ...   
93681  identity theft   smishing sms fraud  creditdeb...   
93682  received call from  number asking about phone ...   
93683  cyber stalking   blackmailing   phonesmsvoip c...   
93684  call karke bola ki aapka lotary laga ha aru ac...   
93685  there is app name koko loan app they send the ...   

                                        word_frequencies  
0      [i, had, continue, received, random, calls, an...  
1      [the, above, fraudster, is, continuously, mess...  
2      [he, is, acting, like, a, police, and, demandi...  
3      [in, apna, job, i, have, applied, fo

In [5]:
train_fraud

,category,sub_category,crimeaditionalinfo,word_frequencies
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,i had continue received random calls and abusi...,"[i, had, continue, received, random, calls, an..."
1,Online Financial Fraud,Fraud CallVishing,the above fraudster is continuously messaging ...,"[the, above, fraudster, is, continuously, mess..."
2,Online Gambling Betting,Online Gambling Betting,he is acting like a police and demanding for m...,"[he, is, acting, like, a, police, and, demandi..."
3,Online and Social Media Related Crime,Online Job Fraud,in apna job i have applied for job interview f...,"[in, apna, job, i, have, applied, for, job, in..."
4,Online Financial Fraud,Fraud CallVishing,i received a call from lady stating that she w...,"[i, received, a, call, from, lady, stating, th..."
...,...,...,...,...
93681,Online Financial Fraud,Internet Banking Related Fraud,identity theft smishing sms fraud creditdeb...,"[identity, theft, smishing, sms, fraud, credit..."
93682,Online Financial Fraud,EWallet Related Fraud,received call from number asking about phone ...,"[received, call, from, number, asking, about, ..."
93683,Online Financial Fraud,UPI Related Frauds,cyber stalking blackmailing phonesmsvoip c...,"[cyber, stalking, blackmailing, phonesmsvoip, ..."
93684,Online and Social Media Related Crime,Online Matrimonial Fraud,call karke bola ki aapka lotary laga ha aru ac...,"[call, karke, bola, ki, aapka, lotary, laga, h..."


In [6]:
import pandas as pd
from collections import defaultdict, Counter
import re
from nltk.corpus import stopwords
import re
crime_keywords = {
    "Women/Child-Related Crime": {
        "Child Pornography/Child Sexual Abuse Material (CSAM)": {
            "keywords": {"child pornography", "csam", "child exploitation", "child sexual abuse", "child porn"}
        },
        "Rape/Gang Rape-Sexually Abusive Content": {
            "keywords": {"rape", "gang rape", "sexual abuse", "sexual assault", "forced sex"}
        },
        "Sale, Publishing, and Transmitting Obscene Material/Sexually Explicit Material": {
            "keywords": {"obscene material", "explicit content", "pornography", "sexually explicit", "adult content"}
        }
    },
    "Financial Fraud Crimes": {
        "Debit/Credit Card Fraud": {
            "keywords": {"credit ","card fraud", "debit"," card fraud", "card scam", "card theft"}
        },
        "SIM Swap Fraud": {
            "keywords": {"sim swap fraud", "sim hacking", "sim card fraud","sim"}
        },
        "Internet Banking-Related Fraud": {
            "keywords": {"internet banking fraud", "online banking fraud", "banking scam","bank","money","loan","lending"}
        },
        "Business Email Compromise/Email Takeover": {
            "keywords": {"email takeover", "email compromise", "business email hack"}
        },
        "E-Wallet Related Frauds": {
            "keywords": {"e-wallet fraud", "digital wallet fraud", "mobile wallet scam"}
        },
        "Fraud Call/Vishing": {
            "keywords": {"vishing", "fraud call", "fake call", "phone scam"}
        },
        "Demat/Depository Fraud": {
            "keywords": {"demat fraud", "depository fraud", "investment account fraud","deposit"}
        },
        "UPI-Related Frauds": {
            "keywords": {"upi fraud", "upi scam", "online payment fraud"," online"}
        },
        "Aadhaar Enabled Payment System (AEPS) Fraud": {
            "keywords": {"aeps fraud", "aadhaar payment fraud", "aadhaar scam"}
        }
    },
    "Cyber Crimes": {
        "Email Phishing": {
            "keywords": {"email phishing", "phishing scam", "phishing email", "email fraud"}
        },
        "Cheating by Impersonation": {
            "keywords": {"impersonation fraud", "fake identity", "identity impersonation"}
        },
        "Fake/Impersonating Profile": {
            "keywords": {"fake profile", "impersonation profile", "fake social profile"}
        },
        "Profile Hacking/Identity Theft": {
            "keywords": {"identity theft", "profile hacking", "identity fraud","fraudster"}
        },
        "Provocative Speech of Unlawful Acts": {
            "keywords": {"provocative speech", "unlawful acts", "incitement", "hate speech"}
        },
        "Intimidating Email": {
            "keywords": {"intimidating email", "threatening email", "email harassment"}
        },
        "Online Job Fraud": {
            "keywords": {"job fraud", "employment scam", "fake job offer","job"}
        },
        "Online Matrimonial Fraud": {
            "keywords": {"matrimonial fraud", "marriage scam", "dating scam","marriage"}
        },
        "Cyber Bullying/Stalking/Sexting": {
            "keywords": {"cyberbullying", "stalking", "sexting", "online harassment"}
        }
    },
    "Cyber Attacks": {
        "Defacement/Hacking": {
            "keywords": {"website defacement", "hacking", "web hacking", "site defacement","money","app"}
        },
        "Unauthorized Access/Data Breach": {
            "keywords": {"unauthorized access", "data breach", "data leak", "breach of privacy"}
        },
        "Ransomware": {
            "keywords": {"ransomware", "ransom virus", "ransom malware"}
        },
        "Cryptocurrency Crime": {
            "keywords": {"cryptocurrency scam", "bitcoin fraud", "crypto theft"}
        },
        "Cyber Terrorism": {
            "keywords": {"cyber terrorism", "cyber attack", "online terrorism"}
        }
    },
    "Network and Infrastructure Attacks": {
        "Denial of Service (DoS) and Distributed Denial of Service (DDoS)": {
            "keywords": {"dos attack", "ddos attack", "denial of service"}
        },
        "Attacks on Critical Infrastructure, SCADA, Operational Technology Systems": {
            "keywords": {"critical infrastructure attack", "scada attack", "ot systems hack"}
        }
    }
}

print(crime_keywords)
train_fraud['Category1'] = ""
train_fraud['Subcategory1'] = ""

# Define a function to classify the text

stop_words = set(stopwords.words('english')).union({"i", "have", "has", "the"})
def split_and_filter(sentence):
    if isinstance(sentence, str):  # Ensure the input is a string
        # Tokenize, remove non-English characters, convert to lowercase
        words = re.findall(r'\b[a-z]+\b', sentence.lower())  # Only words with a-z characters
        # Filter out stopwords
        filtered_words = [word for word in words if word not in stop_words]
        matches = defaultdict(int)
        for category, subcategories in crime_keywords.items():
            for subcategory, keywords in subcategories.items():
                for word in words:
                    if word in keywords:
                        matches[(category, subcategory)] += 1  # Count occurrence in subcategory
        return dict(matches), words
    else:
        # Return an empty dictionary if the sentence is not a string
        return {}, []

# Apply classification to each row in 'additional_info'
train_fraud[[ 'category1','subcategory1']] = train_fraud['crimeaditionalinfo'].apply(
    lambda x: pd.Series(split_and_filter(x))
)


# Display the updated DataFrame with classifications
print(train_fraud)

{'Women/Child-Related Crime': {'Child Pornography/Child Sexual Abuse Material (CSAM)': {'keywords': {'csam', 'child sexual abuse', 'child porn', 'child exploitation', 'child pornography'}}, 'Rape/Gang Rape-Sexually Abusive Content': {'keywords': {'sexual assault', 'forced sex', 'rape', 'sexual abuse', 'gang rape'}}, 'Sale, Publishing, and Transmitting Obscene Material/Sexually Explicit Material': {'keywords': {'explicit content', 'adult content', 'sexually explicit', 'obscene material', 'pornography'}}}, 'Financial Fraud Crimes': {'Debit/Credit Card Fraud': {'keywords': {'credit ', 'card theft', 'card fraud', ' card fraud', 'debit', 'card scam'}}, 'SIM Swap Fraud': {'keywords': {'sim swap fraud', 'sim', 'sim hacking', 'sim card fraud'}}, 'Internet Banking-Related Fraud': {'keywords': {'loan', 'internet banking fraud', 'bank', 'banking scam', 'lending', 'money', 'online banking fraud'}}, 'Business Email Compromise/Email Takeover': {'keywords': {'email takeover', 'email compromise', 'bus

In [7]:
import pandas as pd
import re
from nltk.corpus import stopwords
from collections import defaultdict
stop_words = set(stopwords.words('english')).union({"i", "have", "has", "the"})
def split_and_filter(sentence):
    if isinstance(sentence, str):  # Ensure the input is a string
        # Tokenize, remove non-English characters, convert to lowercase
        words = re.findall(r'\b[a-z]+\b', sentence.lower())  # Only words with a-z characters
        # Filter out stopwords
        filtered_words = [word for word in words if word not in stop_words]
        matches = defaultdict(int)
        for category, subcategories in crime_keywords.items():
            for subcategory, keywords in subcategories.items():
                for word in filtered_words:
                    if word in keywords:
                        matches[(category, subcategory)] += 1  # Count occurrence in subcategory
        return dict(matches), filtered_words
    else:
        # Return an empty dictionary if the sentence is not a string
        return {}, []



train_fraud[[ 'matching_words','filtered_words']] = train_fraud['crimeaditionalinfo'].apply(
    lambda x: pd.Series(split_and_filter(x))
)

# To count the total matched keywords frequency for each row
#train_fraud['total_keyword_frequency'] = train_fraud['matched_keyword_counts'].apply(lambda x: sum(x.values()))

# Display the DataFrame with filtered words and keyword frequency columns
print(train_fraud[['crimeaditionalinfo', 'filtered_words']])

                                      crimeaditionalinfo  \
0      i had continue received random calls and abusi...   
1      the above fraudster is continuously messaging ...   
2      he is acting like a police and demanding for m...   
3      in apna job i have applied for job interview f...   
4      i received a call from lady stating that she w...   
...                                                  ...   
93681  identity theft   smishing sms fraud  creditdeb...   
93682  received call from  number asking about phone ...   
93683  cyber stalking   blackmailing   phonesmsvoip c...   
93684  call karke bola ki aapka lotary laga ha aru ac...   
93685  there is app name koko loan app they send the ...   

                                          filtered_words  
0      [continue, received, random, calls, abusive, m...  
1      [fraudster, continuously, messaging, asking, p...  
2      [acting, like, police, demanding, money, addin...  
3      [apna, job, applied, job, interview,

In [8]:
train_fraud.columns

Index(['category', 'sub_category', 'crimeaditionalinfo', 'word_frequencies',
       'Category1', 'Subcategory1', 'category1', 'subcategory1',
       'matching_words', 'filtered_words'],
      dtype='object')

In [9]:
train= train_fraud[['category', 'sub_category', 'crimeaditionalinfo',
        'filtered_words']]

In [10]:
train

,category,sub_category,crimeaditionalinfo,filtered_words
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,i had continue received random calls and abusi...,"[continue, received, random, calls, abusive, m..."
1,Online Financial Fraud,Fraud CallVishing,the above fraudster is continuously messaging ...,"[fraudster, continuously, messaging, asking, p..."
2,Online Gambling Betting,Online Gambling Betting,he is acting like a police and demanding for m...,"[acting, like, police, demanding, money, addin..."
3,Online and Social Media Related Crime,Online Job Fraud,in apna job i have applied for job interview f...,"[apna, job, applied, job, interview, telecalli..."
4,Online Financial Fraud,Fraud CallVishing,i received a call from lady stating that she w...,"[received, call, lady, stating, send, new, pho..."
...,...,...,...,...
93681,Online Financial Fraud,Internet Banking Related Fraud,identity theft smishing sms fraud creditdeb...,"[identity, theft, smishing, sms, fraud, credit..."
93682,Online Financial Fraud,EWallet Related Fraud,received call from number asking about phone ...,"[received, call, number, asking, phone, pay, c..."
93683,Online Financial Fraud,UPI Related Frauds,cyber stalking blackmailing phonesmsvoip c...,"[cyber, stalking, blackmailing, phonesmsvoip, ..."
93684,Online and Social Media Related Crime,Online Matrimonial Fraud,call karke bola ki aapka lotary laga ha aru ac...,"[call, karke, bola, ki, aapka, lotary, laga, h..."


In [18]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_fraud['filtered_words']=train_fraud['filtered_words'].apply(lambda x: ' '.join(map(str, x)) if isinstance(x, list) else str(x))

# Apply label encoding to the 'Category' column
train_fraud['category'] = label_encoder.fit_transform(train_fraud['category'])
train_fraud['sub_category'] = label_encoder.fit_transform(train_fraud['sub_category'])
train_fraud['filtered_words'] = label_encoder.fit_transform(train_fraud['filtered_words'])


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# Sample data loading (replace this with your dataset)
# Assuming 'fraud' is the target variable
# train_fraud = pd.read_csv('your_fraud_data.csv')
# X = train_fraud.drop('fraud', axis=1)  # Replace 'fraud' with your target column
# y = train_fraud['fraud']

# Example data for demonstration
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define models
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Support Vector Machine": SVC(probability=True),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Neural Network": MLPClassifier()
}

# Train and evaluate each model
results = []

for model_name, model in models.items():
    # Train model
    model.fit(X_train, y_train)
    
    # Predict on test data
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob) if y_prob is not None else None
    
    # Store results
    results.append({
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "ROC AUC": roc_auc
    })

# Convert results to DataFrame for easy comparison
results_df = pd.DataFrame(results)
print(results_df.sort_values(by="F1 Score", ascending=False))


                    Model  Accuracy  Precision    Recall  F1 Score   ROC AUC
2           Random Forest  0.896667   0.942029  0.849673  0.893471  0.963385
3       Gradient Boosting  0.880000   0.939850  0.816993  0.874126  0.961540
1           Decision Tree  0.866667   0.879195  0.856209  0.867550  0.866880
0     Logistic Regression  0.856667   0.871622  0.843137  0.857143  0.942110
4  Support Vector Machine  0.856667   0.898551  0.810458  0.852234  0.926771
6          Neural Network  0.853333   0.892086  0.810458  0.849315  0.940687
5     K-Nearest Neighbors  0.813333   0.881890  0.732026  0.800000  0.899093


C:\Users\ss529\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
